## start

In [1]:
from google.colab import files
from google.colab import drive
drive.mount('/content/mount')

DRIVE_WORKSPACE = '/content/mount/My\ Drive/00_work/00_src_models'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/mount


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
!unzip protobuf.zip

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [2]:
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information

cat: '/proc/driver/nvidia/gpus/0000:00:04.0/information': No such file or directory


### Dependencies

In [0]:
!python 'object_detection/builders/model_builder_test.py'

### Load model

In [0]:
DRIVE_WORKSPACE = '/content/mount/My\ Drive/00.work'
SRC_PATH=DRIVE_WORKSPACE+'/content/mount/My\ Drive/00.work/00_src_models/research'
%cd {SRC_PATH}
!pwd
DEST_DIR = CUR_DIR + '/' + 'pretrained_model'
LATEST_CHECKPOINT = '/content/mount/My\ Drive/DogCatDetection/last_checkpoint'

In [0]:
!ls {LATEST_CHECKPOINT}

In [0]:
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
    
!mkdir {DEST_DIR} && cp -a {LATEST_CHECKPOINT}/. {DEST_DIR}

### Data Preparation

In [0]:
import tensorflow as tf

PATH_TO_LABELS = '/content/mount/My Drive/00.work/00_src_models/research/01_configfile/pascal_label_map_sphygmomanometer.pbtxt'

detection_graph = tf.Graph()
with detection_graph.as_default():
    with tf.gfile.GFile(PATH_TO_LABELS) as fid:
        label_map_string = fid.read()
    
    with tf.gfile.GFile(PATH_TO_LABELS, 'w') as fid: # protobuf text format under created /modelInference/ folder        
        print(label_map_string)
        # fid.write(label_map_string)

In [0]:
DATA_PATH = '/content/mount/My\ Drive/00.work/finetune_data'
!ls {DATA_PATH}

In [0]:
!cp {DATA_PATH}/annotations.tar.gz ./
!cp {DATA_PATH}/images.tar.gz ./
!tar -xvf annotations.tar.gz
!tar -xvf images.tar.gz

In [0]:
#train data
PATH_TO_LABELS = '/content/mount/My\ Drive/00.work/00_src_models/research/01_configfile/pascal_label_map_sphygmomanometer.pbtxt'
!python object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path={PATH_TO_LABELS}  \
    --data_dir='/content/mount/My Drive/00.work/finetune_data' --year=VOC2019 --set=train \
    --output_path='/content/mount/My Drive/00.work/finetune_data/pascal_train_sphygmomanometer.record'

In [0]:
#valuation data
PATH_TO_LABELS = '/content/mount/My\ Drive/00.work/00_src_models/research/01_configfile/pascal_label_map_sphygmomanometer.pbtxt'
!python object_detection/dataset_tools/create_pascal_tf_record.py \
    --label_map_path={PATH_TO_LABELS}  \
    --data_dir='/content/mount/My Drive/00.work/finetune_data' --year=VOC2019 --set=val \
    --output_path='/content/mount/My Drive/00.work/finetune_data/pascal_val_sphygmomanometer.record'

## Pipeline

In [0]:
import tensorflow as tf
from object_detection.utils import label_map_util
import re

%matplotlib inline

In [0]:
test_record_fname = '/content/Finetune_SSD_MobileNetV2/models/research/pet_faces_train.record-?????-of-00010'
train_record_fname = '/content/Finetune_SSD_MobileNetV2/models/research//pet_faces_val.record-?????-of-00010'
label_map_pbtxt_fname = '/content/Finetune_SSD_MobileNetV2/models/research/object_detection/data/pet_label_map.pbtxt'

In [0]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/Finetune_SSD_MobileNetV2/models/research/pretrained_model/model.ckpt'

In [0]:
pipeline_fname = os.path.join('/content/Finetune_SSD_MobileNetV2/models/research/pretrained_model/', 'pipeline.config')
pipeline_fname

'/content/Finetune_SSD_MobileNetV2/models/research/pretrained_model/pipeline.config'

In [0]:
def get_num_classes(pbtxt_fname):
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [0]:
num_classes = get_num_classes(label_map_pbtxt_fname)
metric_eval = 'oid_V2_detection_metrics'
batch_size = 64
num_steps = 1000
use_checkpoint = 'true'
init_lr = 1e-6
decay_steps = 10000
load_detect_ckpt = 'true'
optimizer = 'adam_optimizer'

with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    s = re.sub('from_detection_checkpoint: "[a-z]+"',
               'from_detection_checkpoint: {}'.format(use_checkpoint), s)
    s = re.sub('load_all_detection_checkpoint_vars: "[a-z]+"',
               'load_all_detection_checkpoint_vars: {}'.format(load_detect_ckpt), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    # Set metric of evaluation
    s = re.sub(
        'metrics_set: ".*?"', 
        'metrics_set: "{}"'.format(metric_eval), s)
    
    # Set initial learning rate & weight decays
    s = re.sub('initial_learning_rate: [0-9]+[\.][0-9]+[e][-][0-9]+',
               'initial_learning_rate: {}'.format(init_lr), s)
    s = re.sub('decay_steps: [0-9]+',
               'decay_steps: {}'.format(decay_steps), s)
    
    # Change optimizer to Adam
    s = re.sub('rms_prop_optimizer',
           '{}'.format(optimizer), s)
    
    s = re.sub('momentum_optimizer_value: 0.8999999761581421', '', s)
    s = re.sub('decay: 0.8999999761581421', '', s)
    s = re.sub('epsilon: 1.0', '', s)
    
    f.write(s)

In [0]:
!cat {pipeline_fname}

In [0]:
MODEL_DIR = DEST_DIR + '/' + 'training'
# Optionally remove content in output model directory to fresh start.
!rm -rf {MODEL_DIR}
os.makedirs(MODEL_DIR, exist_ok=True)

get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = MODEL_DIR
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://416565a6.ngrok.io


In [0]:
!ls {LOG_DIR}

In [0]:
!pip install -q pycocotools

In [0]:
!pip install albumentations
!pip install pascal_voc_writer

In [4]:
!git clone  https://github.com/yuslzq/sys_oper.git

Cloning into 'sys_oper'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 2590 (delta 2), reused 6 (delta 2), pack-reused 2582
Receiving objects: 100% (2590/2590), 128.75 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (512/512), done.
Checking out files: 100% (2495/2495), done.


In [0]:
%cd /content/sys_oper
!git pull

In [0]:
!git clone https://github.com/yuslzq/00_src_models.git

```
!git clone https://github.com/google-research/tf-slim
!mkdir /content/sys_oper/00_src2/research/tf_slim
!cp -r tf-slim/tf_slim/* /content/sys_oper/00_src2/research/tf_slim/
!pip install tf_slim
!tf_upgrade_v2 \
  --intree sys_oper/00_src/ \
  --outtree sys_oper/00_src2/ \
  --reportfile report.txt
```



In [0]:
!pip list

```
tf2.0
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod2/local_installers/cuda-repo-ubuntu1710-9-2-local_9.2.148-1_amd64
!dpkg -i cuda-repo-ubuntu1710-9-2-local_9.2.148-1_amd64
# You can use this line to find out the directory name
# !ls /var/ | grep cuda-repo
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda
****
tf 1.3
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
```



In [0]:
import os
print(os.environ['PYTHONPATH'] )

In [1]:
!pip install numpy==1.17.4

## Training

In [0]:

!pip install tensorflow==1.15.0
!pip install tensorflow-gpu==1.15.0
!pip install numpy==1.17.4
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
!pip  uninstall tensorboard
!git clone  https://github.com/yuslzq/sys_oper.git

NUM_TRAIN_STEPS = 1000000
SAMPLE_1_OF_N_EVAL_EXAMPLES = 1
PIPELINE_FNAME = '/content/sys_oper/01_configfile/00_ssdM2.config'
MODEL_DIR = '/content/mount/My\ Drive/00_work/ssd_ysl_ssdM2_sysoper'
import os
os.environ['PYTHONPATH'] = '/env/python:/content/sys_oper/00_src/research/:/content/sys_oper/00_src/research/slim/'

!python /content/sys_oper/00_src/research/object_detection/model_main.py \
    --pipeline_config_path={PIPELINE_FNAME} \
    --model_dir={MODEL_DIR} \
    --num_train_steps={NUM_TRAIN_STEPS} \
    --sample_1_of_n_eval_examples={SAMPLE_1_OF_N_EVAL_EXAMPLES} \
    --alsologtostderr

In [0]:
!nvcc -v 

## Export

# make pb file

In [0]:
CONFIG_FILE='/content/mount/My\ Drive/00_work/00_src_models/research/01_configfile/rcnn_david_ssdMv2_fpnlite300RgbFigColab.config'
EXPORT_OUTPUT_DIR='/content/mount/My\ Drive/00_work/ssdM2Fpn300_err/tflite'
CHECKPOINT_PATH='/content/mount/My\ Drive/00_work/ssdM2Fpn300_err/model.ckpt-48950'
!python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path=$CONFIG_FILE \
--trained_checkpoint_prefix=$CHECKPOINT_PATH \
--output_directory=$EXPORT_OUTPUT_DIR \
--add_postprocessing_op=true

#make tflite file

In [0]:
PATH_TO_MODEL='/content/mount/My\ Drive/00_work/ssdM2Fpn300_err/tflite'
!tflite_convert    \
 --graph_def_file={PATH_TO_MODEL}/tflite_graph.pb \
 --output_file={PATH_TO_MODEL}/detectfpn300.tflite \
 --input_shapes=1,300,300,3 \
 --output_format=TFLITE \
 --input_arrays=normalized_input_image_tensor \
 --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
 --inference_type=FLOAT \
 --mean_values=128 \
 --std_dev_values=128 \
 --change_concat_input_ranges=false \
 --allow_custom_ops

In [0]:
import numpy as np
import shutil

In [0]:
output_directory = './fine_tuned_model'

if os.path.exists(output_directory):
    shutil.rmtree(output_directory)
    
lst = os.listdir(MODEL_DIR)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps = np.array([int(re.findall('\d+',l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(MODEL_DIR, last_model)
print(last_model_path)

!python /content/Finetune_SSD_MobileNetV2/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [0]:
log = os.listdir(MODEL_DIR)
log = [l for l in log if 'events.out.tfevents.' in l]

for item in log:
    print(output_directory)
    shutil.copy(os.path.join(MODEL_DIR, item), output_directory)

./fine_tuned_model


In [0]:
DRIVE_WORKSPACE = '/content/mount/My\ Drive/DogCatDetection'
LATEST_CHECKPOINT = '/content/mount/My\ Drive/DogCatDetection/last_checkpoint'
FOLDER_CHECKPOINT = '/content/mount/My\ Drive/DogCatDetection/'+'train_' # setting number here
# Function to back up all checkpoint: Folder train + number
arr = []
!ls {DRIVE_WORKSPACE}
arr = !ls {DRIVE_WORKSPACE}
swap = []

def getFolder(arr):
    idx = []
    for elem in arr:
        if elem[:5] == 'train':
            idx.append(int(elem[6:]))
    return idx

if len(arr) == 0: 
    FOLDER_CHECKPOINT = FOLDER_CHECKPOINT + '{}'.format(1) # create 1st train directory
    !cp -a {output_directory}/. {FOLDER_CHECKPOINT} # copy output directory to drive
else:
    for small_arr in arr:
        for elem in small_arr.split():
            swap.append(elem)
    arr = swap

    idx = getFolder(arr)
    print(idx)   
    print('Last trained folder: {}'.format(max(idx)))
    FOLDER_CHECKPOINT = FOLDER_CHECKPOINT + '{}'.format(int(max(idx))+1) # create new train directory
    !cp -a {output_directory}/. {FOLDER_CHECKPOINT} # copy output directory to drive

arr = []
arr = !ls {DRIVE_WORKSPACE}
swap = []

for small_arr in arr:
    for elem in small_arr.split():
        swap.append(elem)
arr = swap
idx = getFolder(arr)
print(idx) 
print('Number of trained folder: {}'.format(len(idx)))
print('New trained folder: {}'.format(max(idx)))

!ls {DRIVE_WORKSPACE}

label_map	 test_img  train_10  train_2  train_4  train_6	train_8
last_checkpoint  train_1   train_11  train_3  train_5  train_7	train_9
[10, 2, 4, 6, 8, 1, 11, 3, 5, 7, 9]
Last trained folder: 11
[1, 12, 4, 7, 10, 2, 5, 8, 11, 3, 6, 9]
Number of trained folder: 12
New trained folder: 12
label_map	 train_1   train_12  train_4  train_7
last_checkpoint  train_10  train_2   train_5  train_8
test_img	 train_11  train_3   train_6  train_9


In [0]:
!test -e {LATEST_CHECKPOINT} && rm -rf {LATEST_CHECKPOINT} || echo LATEST_CHECKPOINT not found && mkdir {LATEST_CHECKPOINT} # create new ckpt
!cd {DRIVE_WORKSPACE} && ls 
!cp -a {output_directory}/. {LATEST_CHECKPOINT}
!cd {LATEST_CHECKPOINT} && ls 

## Inference

# 新しいセクション

In [0]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import glob

%matplotlib inline

In [0]:
!mkdir 'modelInference'
%cd 'modelInference'
!mkdir 'test_images' && cd 'test_images' && pwd
!cd 'test_images' && wget 'https://www.petmd.com/sites/default/files/Acute-Dog-Diarrhea-47066074.jpg'
!cd 'test_images' && wget 'https://cdn-images-1.medium.com/max/1600/1*mONNI1lG9VuiqovpnYqicA.jpeg'
!cd 'test_images' && wget 'https://www.lifewithdogs.tv/wp-content/uploads/2015/01/gfhdgfgfgdfgdf.jpg'

In [0]:
!cp {LATEST_CHECKPOINT}/frozen_inference_graph.pb ./
!cp {PATH_TO_LABELS} ./
!ls

cp: '/content/Finetune_SSD_MobileNetV2/models/research/modelInference/pet_label_map.pbtxt' and './pet_label_map.pbtxt' are the same file
frozen_inference_graph.pb  pet_label_map.pbtxt	test_images


In [0]:
PATH_TO_FROZEN_GRAPH = os.path.join(os.path.abspath('/content/Finetune_SSD_MobileNetV2/models/research/modelInference'), "frozen_inference_graph.pb")
assert os.path.isfile(PATH_TO_FROZEN_GRAPH), '`{}` not exist'.format(PATH_TO_FROZEN_GRAPH)
print(PATH_TO_FROZEN_GRAPH)

PATH_TO_LABELS = os.path.join(os.path.abspath('/content/Finetune_SSD_MobileNetV2/models/research/modelInference'), "pet_label_map.pbtxt")
assert os.path.isfile(PATH_TO_LABELS), '`{}` not exist'.format(PATH_TO_LABELS)
print(PATH_TO_LABELS)

/content/Finetune_SSD_MobileNetV2/models/research/modelInference/frozen_inference_graph.pb
/content/Finetune_SSD_MobileNetV2/models/research/modelInference/pet_label_map.pbtxt


In [0]:
PATH_TO_TEST_IMAGES_DIR = '/content/Finetune_SSD_MobileNetV2/models/research/modelInference/test_images'
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
TEST_IMAGE_PATHS

['/content/Finetune_SSD_MobileNetV2/models/research/modelInference/test_images/Acute-Dog-Diarrhea-47066074.jpg',
 '/content/Finetune_SSD_MobileNetV2/models/research/modelInference/test_images/1*mONNI1lG9VuiqovpnYqicA.jpeg',
 '/content/Finetune_SSD_MobileNetV2/models/research/modelInference/test_images/gfhdgfgfgdfgdf.jpg']

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)

In [0]:
num_classes = get_num_classes(PATH_TO_LABELS)
num_classes

39

In [0]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    print(image_path)
    print(output_dict['detection_classes'][:5])
    print(output_dict['detection_scores'][:5])
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)